In [20]:
# clear all the variables
%reset -f
import numpy as np
import pandas as pd
import sys
import turicreate as tc
import os
from os import walk
import scipy.io
import xlrd, os, fnmatch
import array
import pandas as pd
import tensorflow as tf
import plotly.plotly as py
import plotly.graph_objs as go
from scipy.spatial.distance import cdist
# Path variable (where the data is located)
myPath = './data/RSVP_spatial_cue_replication/' 
# Append the path variable to existing search path
sys.path.append(myPath)
# Get the file information in the directory
file_list = []
ignore_list = ['pilot']
for root, dirs, files in os.walk(myPath):
    if 'pilot' in dirs:
        dirs[:]=[]
    for filename in files:
        if fnmatch.fnmatch(filename.lower(), "*csv"):
            file_list.append(os.path.join(root, filename))

In [21]:
file_list

['./data/RSVP_spatial_cue_replication/20_07Feb2019_12-10.csv',
 './data/RSVP_spatial_cue_replication/24_07Feb2019_17-30.csv',
 './data/RSVP_spatial_cue_replication/3_05Feb2019_17-54.csv',
 './data/RSVP_spatial_cue_replication/30_19Feb2019_09-35.csv',
 './data/RSVP_spatial_cue_replication/9_06Feb2019_10-34.csv',
 './data/RSVP_spatial_cue_replication/10_06Feb2019_11-01.csv',
 './data/RSVP_spatial_cue_replication/26_08Feb2019_17-26.csv',
 './data/RSVP_spatial_cue_replication/15_06Feb2019_17-02.csv',
 './data/RSVP_spatial_cue_replication/18_06Feb2019_18-35.csv',
 './data/RSVP_spatial_cue_replication/2_05Feb2019_17-26.csv',
 './data/RSVP_spatial_cue_replication/27_08Feb2019_18-00.csv',
 './data/RSVP_spatial_cue_replication/14_06Feb2019_14-42.csv',
 './data/RSVP_spatial_cue_replication/16_06Feb2019_17-35.csv',
 './data/RSVP_spatial_cue_replication/6_05Feb2019_19-29.csv',
 './data/RSVP_spatial_cue_replication/5_05Feb2019_19-03.csv',
 './data/RSVP_spatial_cue_replication/29_08Feb2019_18-57.csv

In [22]:
data = tc.SFrame.read_csv(file_list[0])
print(len(data))
temp = data
for i in range(1,len(file_list)):
    temp = tc.SFrame.read_csv(file_list[i], column_type_hints = [str,int,int,str,int,int,str,str,int,int,array.array,int,int])
    #print(len(temp))
    
    print(zip(temp.column_types(),temp.column_names()))
    data = data.append(temp)
len(data)

Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/RSVP_spatial_cue_replication/20_07Feb2019_12-10.csv

Parsing completed. Parsed 100 lines in 0.038015 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,str,int,int,str,str,int,int,array,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/RSVP_spatial_cue_replication/20_07Feb2019_12-10.csv

Parsing completed. Parsed 280 lines in 0.009158 secs.

...

Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/RSVP_spatial_cue_replication/4_05Feb2019_18-31.csv

Parsing completed. Parsed 280 lines in 0.007904 secs.

Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/RSVP_spatial_cue_replication/28_08Feb2019_18-32.csv

Parsing completed. Parsed 280 lines in 0.006953 secs.

8400

In [23]:
len(data)/280

30.0

In [24]:
from plotly import tools
import colorlover as cl
trace = [[]]*2
ecc_length = len(data['cueEccentricity1'].unique())
colors = cl.scales['3']['seq']['YlOrRd']
for i,cuePos in enumerate(data['cueSpatialPos1'].unique()):
    cuePosdata = data[data['cueSpatialPos1']==cuePos]
    temp1 = []
    for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        temp1.append(go.Histogram(x = ecc_data['responsePosRelative0'].to_numpy(), 
                                  histnorm='probability',name = str(ecc) + 'deg', opacity = 1, marker = dict(color = colors[j])))
    trace[i] = temp1

if data['cueSpatialPos1'].unique()[0] == [-1.0,0.0]:
    titles = ('West','East')
else:
    titles = ('East','West')
    
fig = tools.make_subplots(rows=1, cols=2, subplot_titles = titles)
fig['layout'].update(barmode='group', title = 'performance in RSVP visual now task for different cue positions - '+str(int(len(data)/280))+' participants')
#fig['layout']['xaxis3'].update(title='position of reported item relative to the true item', range = [-2,2])
#fig['layout']['xaxis4'].update(title='position of reported item relative to the true item', range = [-2,2])
fig['layout']['xaxis1'].update(title='position of reported item relative to the true item')
fig['layout']['xaxis2'].update(title='position of reported item relative to the true item')
fig['layout']['yaxis1'].update(title='Probability')
#fig['layout']['yaxis3'].update(title='Frequency')
for i in range(ecc_length):
    fig.add_trace(trace[0][ecc_length-1-i], 1, 1)
    fig.add_trace(trace[1][ecc_length-1-i], 1, 2)
    #fig.append_trace(trace[2][i], 2, 1)
    #fig.append_trace(trace[3][i], 2, 2)
py.iplot(fig, filename='RSVP_visual_now')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [25]:
trace_data = []
for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        trace_data.append(go.Histogram(x = ecc_data['responsePosRelative0'].to_numpy(), 
                                  histnorm='probability',name = str(ecc) + 'deg', opacity = 1, marker = dict(color = colors[j])))

        
fig = tools.make_subplots(rows=1, cols=1)
fig['layout'].update(barmode='group', title = 'performance in RSVP visual now task for different cue positions - '+str(int(len(data)/280))+' participants', height=600,width = 650)
fig['layout']['xaxis1'].update(range = [-3,3], title='position of reported item relative to the true item')
fig['layout']['yaxis1'].update(title='Probability', range = [0,0.8])
for i in range(ecc_length):
    fig.add_trace(trace_data[ecc_length -1 - i], 1, 1)
py.iplot(fig, filename='RSVP_visual_now_grouped')

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [26]:
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

In [27]:
X_plot = np.linspace(-3, 3, 1000)
trace_data = []
distributions = []
fill_colors = ['rgba(255,237,160,0.2)', 'rgba(254,178,76,0.2)', 'rgba(240,59,32,0.2)']
for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        X = ecc_data['responsePosRelative0'].to_numpy()
        # Gaussian KDE
        #kde = KernelDensity(kernel='gaussian').fit(X)
        kde = gaussian_kde(X)
        log_dens = kde(X_plot)
        distributions.append({'ecc' : ecc, 'dist' : X, 'dens': log_dens})
        trace_data.append(go.Scatter(x = X_plot, y = log_dens,
                                  mode='lines', fill = 'tozeroy', fillcolor = fill_colors[j], name = str(ecc) + 'deg', line = dict(color = colors[j], width = 2) ))
        
fig = tools.make_subplots(rows=1, cols=1)
fig['layout'].update(barmode='group', title = 'performance in RSVP visual now task for different cue positions - '+str(int(len(data)/280))+' participants')
fig['layout']['xaxis1'].update(title='position of reported item relative to the true item')
fig['layout']['yaxis1'].update(title='Probability', range = [0,0.8])
for i in range(ecc_length):
    fig.add_trace(trace_data[i], 1, 1)
py.iplot(fig, filename='RSVP_visual_now_grouped')


This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [28]:
import numpy as np
from scipy.stats import kurtosis, skew

for x in distributions:
    print( 'Eccentricity: {}'.format( x['ecc']))
    print( 'excess kurtosis of normal distribution (should be 0): {}'.format( kurtosis(x['dens']) ))
    print( 'skewness of normal distribution (should be 0): {}'.format( skew(x['dens']) ))
    print( '\n')

Eccentricity: 2
excess kurtosis of normal distribution (should be 0): -0.6554993076569176
skewness of normal distribution (should be 0): 0.8731114004704202


Eccentricity: 10
excess kurtosis of normal distribution (should be 0): -1.2936218044404075
skewness of normal distribution (should be 0): 0.43495197011333436




In [29]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

hsb = data.to_dataframe()
mod = ols('responsePosRelative0 ~ C(cueEccentricity1)',
                data=hsb).fit()
                
aov_table = sm.stats.anova_lm(mod, typ=2)
print(aov_table)

                           sum_sq      df         F    PR(>F)
C(cueEccentricity1)      2.105833     1.0  0.286611  0.592415
Residual             61703.059762  8398.0       NaN       NaN


In [30]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[1]['ecc']))
scipy.stats.ttest_ind(distributions[0]['dist'], distributions[1]['dist'],axis =0, equal_var=True)

Ecc:2 vs.Ecc:10


Ttest_indResult(statistic=-0.07865734852461834, pvalue=0.9373089201261349)

In [31]:
# print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_ind(distributions[0]['dist'], distributions[2]['dist'], axis =0,equal_var=True)

In [32]:
# print('Ecc:{}'.format(distributions[1]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_ind(distributions[1]['dist'], distributions[2]['dist'], axis =0,equal_var=True)

In [33]:
import turicreate.aggregate as agg
sub_data = data.groupby(['subject','cueEccentricity1'],operations = {'avg_lag':agg.AVG('responsePosRelative0')})
ratios = []
c_val = [-1,1]
for sub in data['subject'].unique().sort():
    temp1 = data[data['subject']==sub]
    for ecc in temp1['cueEccentricity1'].unique().sort():
        temp = temp1[temp1['cueEccentricity1']==ecc]
        count_data = temp['responsePosRelative0'].value_counts()
        a = count_data[count_data['value']==c_val[0]]['count'].to_numpy()
        b = count_data[count_data['value']==c_val[1]]['count'].to_numpy()
        if len(a) > 0 and len(b)>0:
            ratios.extend(a-b)
        elif len(a)==0 and len(b)>0:
            ratios.extend(0-b)
        elif len(a)>0 and len(b)==0:
            ratios.extend(a-0)
        else:
            ratios.extend(0)
            
sub_data = sub_data.sort(['subject','cueEccentricity1'])
skewness = []
kurtosissy = []
for sub in data['subject'].unique().sort():
    subd = data[data['subject']==sub]
    for ecc in subd['cueEccentricity1'].unique().sort():
        eccd = subd[subd['cueEccentricity1']==ecc]
        X = eccd['responsePosRelative0'].to_numpy()
        kde = gaussian_kde(X)
        log_dens = kde(X_plot)
        skewness.append(skew(log_dens))
        kurtosissy.append(kurtosis(log_dens))

sub_data['skew'] = skewness
sub_data['kurtosis'] = kurtosissy
sub_data['ratio'] = ratios

In [34]:
sub_hsb = sub_data.to_dataframe()
#sub_hsb = hsb.groupby(['subject','cueEccentricity1']).apply(skew('responseRelative0'))
mod = ols('skew ~ C(cueEccentricity1)',
                data=sub_hsb).fit()
                
aov_table = sm.stats.anova_lm(mod)
aov_table

,df,sum_sq,mean_sq,F,PR(>F)
C(cueEccentricity1),1.0,1.988248,1.988248,10.267171,0.002202
Residual,58.0,11.231756,0.193651,NaN,NaN


In [35]:
mod1 = ols('kurtosis ~ C(cueEccentricity1)',
                data=sub_hsb).fit()
                
aov_table1 = sm.stats.anova_lm(mod1)
aov_table1

,df,sum_sq,mean_sq,F,PR(>F)
C(cueEccentricity1),1.0,1.727905,1.727905,3.035392,0.086768
Residual,58.0,33.016661,0.569253,NaN,NaN


In [36]:
ecc = sub_data['cueEccentricity1'].unique().sort()
deg2_data = sub_data[sub_data['cueEccentricity1']==ecc[0]]
deg2_data = deg2_data.sort('subject')
#deg6_data = sub_data[sub_data['cueEccentricity1']==6]
#deg6_data = deg6_data.sort('subject')
deg10_data = sub_data[sub_data['cueEccentricity1']==ecc[1]]
deg10_data = deg10_data.sort('subject')

deg2 = deg2_data['avg_lag'].to_numpy()
deg2_ratio = deg2_data['ratio'].to_numpy()
#deg6 = deg6_data['avg_lag'].to_numpy()
#deg6_ratio = deg6_data['ratio'].to_numpy()
deg10 = deg10_data['avg_lag'].to_numpy()
deg10_ratio = deg10_data['ratio'].to_numpy()

In [37]:
len(deg2_ratio)

30

In [38]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs. zero mean')
scipy.stats.ttest_1samp(deg2,0.0)

Ecc:2 vs. zero mean


Ttest_1sampResult(statistic=-1.6914901819791122, pvalue=0.10146392756050099)

In [39]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[1]['ecc']))
scipy.stats.ttest_rel(deg2, deg10)

Ecc:2 vs.Ecc:10


Ttest_relResult(statistic=0.45766270044364027, pvalue=0.6506053777140217)

In [40]:
# print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_rel(deg2,deg10)

In [41]:
# print('Ecc:{}'.format(distributions[1]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_rel(deg6,deg10)

In [42]:
output = tc.SFrame()
output['subject'] = np.arange(1,len(deg2_ratio)+1)
output['avg_lag_2deg'] = deg2
#output['avg_lag_6deg'] = deg6
output['avg_lag_10deg'] = deg10

output['ratio_2deg'] = deg2_ratio
#output['ratio_6deg'] = deg6_ratio
output['ratio_10deg'] = deg10_ratio

In [43]:
output

subject,avg_lag_2deg,avg_lag_10deg,ratio_2deg,ratio_10deg
1,-0.3571428571428571,0.15,15,48
2,0.2428571428571428,-0.6857142857142856,2,17
3,-0.05714285714285714,0.19999999999999998,-4,9
4,-0.1785714285714287,0.46428571428571414,-7,15
5,0.02857142857142849,-0.45000000000000007,8,24
6,0.2642857142857142,0.44999999999999996,-18,-18
7,-0.06428571428571435,-0.3142857142857141,2,16
8,-0.42857142857142855,-0.27857142857142847,10,16
9,-0.1571428571428572,-0.2714285714285715,19,33
10,-0.2214285714285712,-0.20714285714285713,17,28


In [44]:
# print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(6))
# scipy.stats.ttest_rel(output['avg_lag_2deg'].to_numpy(), output['avg_lag_6deg'].to_numpy())

In [45]:
print('Ecc:{}'.format(ecc[0])+' vs.'+ 'Ecc:{}'.format(ecc[1]))
scipy.stats.ttest_rel(output['avg_lag_2deg'].to_numpy(), output['avg_lag_10deg'].to_numpy())

Ecc:2 vs.Ecc:10


Ttest_relResult(statistic=0.45766270044364027, pvalue=0.6506053777140217)

In [46]:
# print('Ecc:{}'.format(6)+' vs.'+ 'Ecc:{}'.format(10))
# scipy.stats.ttest_rel(output['avg_lag_6deg'].to_numpy(), output['avg_lag_10deg'].to_numpy())

In [47]:
# print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(6))
# scipy.stats.ttest_rel(output['ratio_2deg'].to_numpy(), output['ratio_6deg'].to_numpy())

In [48]:
print('Ecc:{}'.format(ecc[0])+' vs.'+ 'Ecc:{}'.format(ecc[1]))
scipy.stats.ttest_rel(output['ratio_2deg'].to_numpy(), output['ratio_10deg'].to_numpy())

Ecc:2 vs.Ecc:10


Ttest_relResult(statistic=-5.3072600258461184, pvalue=1.0799781200299439e-05)

In [ ]:
# print('Ecc:{}'.format(6)+' vs.'+ 'Ecc:{}'.format(10))
# scipy.stats.ttest_rel(output['ratio_6deg'].to_numpy(), output['ratio_10deg'].to_numpy())

In [ ]:
trace_data = []
trace_data.append(go.Bar(x = ['preference to lag -1 over lag 1 responses'],
                        y = [output['ratio_2deg'].mean()],
                        name=str(ecc[0])+' deg',
                        marker = dict(color = colors[0]),
                        error_y = dict(type = 'data',
                                      array = [output['ratio_2deg'].std()/np.sqrt(len(output))],
                                      visible=True
                                      )
                        )
                 )

# trace_data.append(go.Bar(x = ['preference to lag -1 over lag 1 responses'],
#                         y = [output['ratio_6deg'].mean()],
#                         name='6 deg',
#                         marker = dict(color = colors[1]),
#                         error_y = dict(type = 'data',
#                                       array = [output['ratio_6deg'].std()/np.sqrt(14)],
#                                       visible=True
#                                       )
#                         )
#                  )

trace_data.append(go.Bar(x = ['preference to lag -1 over lag 1 responses'],
                        y = [output['ratio_10deg'].mean()],
                        name=str(ecc[1])+' deg',
                        marker = dict(color = colors[2]),
                        error_y = dict(type = 'data',
                                      array = [output['ratio_10deg'].std()/np.sqrt(len(output))],
                                      visible=True
                                      )
                        )
                 )

layout = go.Layout(
    barmode='group',
    title = str(int(len(data)/280))+' participants',
    xaxis = dict(zeroline=False),
    yaxis = dict(title = 'avg. # of responses lag('+str(c_val[0])+') - lag('+str(c_val[1])+')',zeroline=False),
)
fig = go.Figure(data=trace_data, layout=layout)
py.iplot(fig, filename='error-bar-bar')